# 자율주행을 위한 자동차 객체 탐지

## 데이터 셋 준비
자동차 블랙박스 영상을 활용하여 새로운 데이터 셋을 만들어 줍니다. 
1분 짜리 영상을 10 프레임당 하나의 이미지로 추출합니다.

In [ ]:
%cd driving
%pwd

/Users/boajang/Desktop/Project_S4/driving


'/Users/boajang/Desktop/Project_S4/driving'

In [ ]:
# 동영상을 프레임으로 변환하여 저장
import cv2
cap = cv2.VideoCapture('./driving.mp4')

# 비디오 프레임 크기, 전체 프레임수, FPS 등 출력
print('Frame width:', int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
print('Frame height:', int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Frame count:', int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print('FPS:', cap.get(cv2.CAP_PROP_FPS))

count = 0
while cap.isOpened():
    ret, frame = cap.read()
    # frame = cv2.resize(frame, (416,416))
    
    # 10 frame 당 하나씩 이미지 추출
    if int(cap.get(1))%10 == 0:
        cv2.imwrite('./export/images/image%d.jpg'%count, frame)

        count += 1
        
    if count == 179:
        break
        
print(count)
cap.release()

Frame width: 640
Frame height: 360
Frame count: 1799
FPS: 29.970146468887023
179


## 이미지 데이터 세트 준비
이미지 리스트 불러오기

In [ ]:
# 이미지가 몇개인지 판단 및 이미지 리스트 불러오기
from glob import glob

# glob() 함수는 인자로 받은 패턴과 이름이 일치하는 모든 파일과 디렉토리의 리스트를 반환
img_list = glob('./export/images/*.jpg')

print(len(img_list))

179


## LabelImg
LabelImg를 활용하여 이미지에 라벨을 추가합니다. 

In [ ]:
import os
LABELIMG_PATH = './labelimg'
# image labelling을 할 수 있는 github file을 불러온다.
if not os.path.exists(LABELIMG_PATH):
  !mkdir {LABELIMG_PATH}
  !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

Cloning into './labelimg'...
remote: Enumerating objects: 1846, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 1846 (delta 38), reused 55 (delta 19), pack-reused 1759
Receiving objects: 100% (1846/1846), 232.81 MiB | 8.21 MiB/s, done.
Resolving deltas: 100% (1086/1086), done.


In [ ]:
!cd {LABELIMG_PATH} && make qt5py3

pyrcc5 -o libs/resources.py resources.qrc


In [ ]:
# run label image python 
!cd {LABELIMG_PATH} && python labelImg.py 

2021-09-02 23:02:56.390 python[28172:1211290] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
Image:/Users/boajang/Desktop/Project_S4/driving/export/images/image0.jpg -> Annotation:/Users/boajang/Desktop/Project_S4/driving/export/labels/image0.txt


## 이미지 데이터를 train과 val 데이터 나누어줍니다.
학습 데이터와 테스트 데이터로 나누기

In [ ]:
# split data to train and val 
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=42)

print(len(train_img_list),len(val_img_list))

143 36


## YoloV5를 활용하기 위한 .yaml 파일 준비

In [ ]:
%cat ./data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['car']


In [ ]:
# write labels
# 나눈 데이터의 경로를 train.txt 와 val.txt 에 각각 나눠주기
with open('./train.txt','w') as f:
    f.write('\n'.join(train_img_list))
with open('./val.txt','w') as f:
    f.write('\n'.join(val_img_list))

In [ ]:
!pip install pyyaml

     |████████████████████████████████| 253 kB 1.8 MB/s eta 0:00:01


In [ ]:
# data.yaml 안의 경로 나누어진 데이터셋의 경로로 수정
import yaml
# yaml 파일 불러오기
with open('./data.yaml', 'r') as f:
    data = yaml.load(f)

print(data)

# 데이터 수정
data['train'] = './train.txt'
data['val'] = './val.txt'

# yaml 파일 저장
with open('./data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 1, 'names': ['car']}
{'train': './train.txt', 'val': './val.txt', 'nc': 1, 'names': ['car']}


/var/folders/66/xr9_bvqd73v1v5zl975wc5dc0000gn/T/ipykernel_22003/4237087232.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f)


## YoloV5를 활용

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 9310, done.
remote: Total 9310 (delta 0), reused 0 (delta 0), pack-reused 9310
Receiving objects: 100% (9310/9310), 9.63 MiB | 7.83 MiB/s, done.
Resolving deltas: 100% (6477/6477), done.


In [ ]:
# yolov5를 위한 패키지 설치
%cd ./yolov5/
! pip install -r requirements.txt

/Users/boajang/Desktop/Project_S4/driving/yolov5


## 데이터를 YoloV5 파일과 함께

In [ ]:
# copy data 
import shutil
# copy export file
src = './export'
dst = './yolov5/export'
shutil.copytree(src,dst)

'./yolov5/export'

In [ ]:
# copy train.txt
src = './train.txt'
dst = './yolov5/train.txt'
shutil.copy(src,dst)

'./yolov5/train.txt'

In [ ]:
# copy val.txt
src = './val.txt'
dst = './yolov5/val.txt'
shutil.copy(src,dst)

'./yolov5/val.txt'

In [ ]:
# copy data.yaml
src = './data.yaml'
dst = './yolov5/data.yaml'
shutil.copy(src,dst)

'./yolov5/data.yaml'

## Train Data
데이터 학습시키기 
```
--img : 이미지 크기
--batch : 배치 
--epochs: 학습 횟수
--data: yaml 파일
--cfg: 모델 선택 -> yolov5s.yaml 선택
--weight: pre-trained model(전이 학습)의 가중치 선택
--name : 결과 저장할 폴더 
```
학습에 걸리는 시간: 3.936 h

In [ ]:
# run train 
%cd ./yolov5/
!python train.py --img 640 --batch 16 --epochs 50 --data ./data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name driving_yolov5s_results


[Errno 2] No such file or directory: './yolov5/'
/Users/boajang/Desktop/Project_S4/driving/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=./data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=driving_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=30
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.9.0 CPU

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, 

## 학습 시킨 weights와 하이퍼 파라미터 결과 확인하기

In [ ]:
%pwd

'/Users/boajang/Desktop/Project_S4/driving/yolov5'

In [ ]:
%cd yolov5

/Users/boajang/Desktop/Project_S4/driving/yolov5


In [ ]:
# Tensorboard 
%load_ext tensorboard
%tensorboard --logdir ./runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 48695), started 0:04:19 ago. (Use '!kill 48695' to kill it.)

In [ ]:
# 테스트 데이터로 확인해보기 
# 50% 이상일 경우 표시
from IPython.display import Image
import os
 
val_img_path=val_img_list[0]
 
!python detect.py --weights ./runs/train/driving_yolov5s_results/weights/best.pt --img 640 --conf 0.5 --source "{val_img_path}"

detect: weights=['./runs/train/driving_yolov5s_results/weights/best.pt'], source=./export/images/image162.jpg, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.9.0 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
image 1/1 /Users/boajang/Desktop/Project_S4/driving/yolov5/export/images/image162.jpg: 384x640 7 cars, Done. (0.363s)
Results saved to runs/detect/exp2
Done. (0.382s)


## 영상으로 Detect 진행하기 

In [ ]:
%cd ..

/Users/boajang/Desktop/Project_S4/driving


In [ ]:
# copy data.yaml
src = './driving.mp4'
dst = './yolov5/driving.mp4'
shutil.copy(src,dst)

'./yolov5/driving.mp4'

In [ ]:
%cd ./yolov5
!python detect.py --weights ./runs/train/driving_yolov5s_results/weights/best.pt  --source ../driving.mp4

detect: weights=['./runs/train/driving_yolov5s_results/weights/best.pt'], source=../driving.mp4, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.9.0 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
video 1/1 (1/1799) /Users/boajang/Desktop/Project_S4/driving/yolov5/../driving.mp4: 384x640 7 cars, Done. (0.383s)
video 1/1 (2/1799) /Users/boajang/Desktop/Project_S4/driving/yolov5/../driving.mp4: 384x640 7 cars, Done. (0.299s)
video 1/1 (3/1799) /Users/boajang/Desktop/Project_S4/driving/yolov5/../driving.mp4: 384x640 7 cars, Done. (0.291s)
v

## 다른 이미지로 Detect

In [ ]:
!python detect.py --weights ./runs/train/driving_yolov5s_results/weights/best.pt --img 640 --conf 0.5 --source ../1.jpg

detect: weights=['./runs/train/driving_yolov5s_results/weights/best.pt'], source=../1.jpg, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.9.0 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
image 1/1 /Users/boajang/Desktop/Project_S4/driving/yolov5/../1.jpg: 448x640 18 cars, Done. (0.416s)
Results saved to runs/detect/exp4
Done. (0.451s)


In [ ]:
!python detect.py --weights ./runs/train/driving_yolov5s_results/weights/best.pt --img 640 --conf 0.5 --source ../2.jpg

detect: weights=['./runs/train/driving_yolov5s_results/weights/best.pt'], source=../2.jpg, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.9.0 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
image 1/1 /Users/boajang/Desktop/Project_S4/driving/yolov5/../2.jpg: 640x640 6 cars, Done. (0.552s)
Results saved to runs/detect/exp5
Done. (0.560s)
